# 2.5a: 受控量子门与 TensorPureSate 类

单体量子门作用于多比特量子态中的某个量子位上

$|\psi'\rangle = \hat{O}|\psi\rangle \Leftrightarrow \psi'_{s_0...s_n...s_{N-1}} = \sum_s O_{s_ns}\psi_{s_0...s...s_{N-1}}$

![single_qubit_gate_tensor_network_representation](./images/single-qubit-gate-tensor-network-repr.png)

量子门作用到多体态的一般情况

![multiple-single-qubit-gate](./images/multiple-single-qubit-gate.png)

![multiple-multi-qubit-gate](./images/multiple-multi-qubit-gate.png)

## 受控量子门 Controlled Gate

**目标比特或目标位**：被幺正变换作用的量子比特

**控制比特或控制位**：决定是否将幺正变换作用到目标位

### 受控非门(CNOT)

若控制位处于|1⟩，则作用泡利x算符翻转目标位的状态，否则不作任何操作；控制位总是不被操作。

$\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$

$\text{CNOT}|00\rangle = |00\rangle \qquad \langle00|\text{CNOT}|00\rangle = 1$

$\text{CNOT}|01\rangle = |01\rangle \qquad \langle01|\text{CNOT}|01\rangle = 1$

$\text{CNOT}|10\rangle = |11\rangle \qquad \langle10|\text{CNOT}|11\rangle = 1$

$\text{CNOT}|11\rangle = |10\rangle \qquad \langle11|\text{CNOT}|10\rangle = 1$

![cnot-gate](./images/cnot-gate.png)

### 受控酉门(controlled-U)

若控制位处于$|1\rangle$，则作用$U$算符翻转目标位的状态，否则不作任何操作

$$
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & U_{0,0} & U_{0,1} \\
0 & 0 & U_{1,0} & U_{1,1}
\end{bmatrix}
$$

![controlled-U](./images/controlled-U.png)

### 受控门计算方法

在实际程序计算时，我们并不需要写出整个受控门，而是仅需利用上述$\bar{U}$相关的计算，这样可大幅节约计算成本。

例子：考虑4比特量子态，以量子位1为控制比特，以量子位2为目标比特，作用上CNOT门，设作用后量子态系数张量为$\Psi$

1. 指标交换为(顺序为：目标位、其他、控制位)：

    $\psi_{s_0s_1s_2s_3} \xrightarrow{\text{permute}(2,0,3,1)} \psi'_{s_2s_0s_3s_1}$

    $s_2$ 是目标位，$s_1$ 是控制位，$s_0$ 和 $s_3$ 是其他位

2. 切片+矩阵化+矩阵乘，作用算符$\sigma^x$：

    $\sigma^x(\psi'_{:,:,:,1})_{[0]} \xrightarrow{\text{reshape}(2,2,2)} \phi$

    1. 切片：$\psi'_{:,:,:,1}$ 取出控制位为1的部分
    2. 矩阵化：$\psi'_{:,:,:,1}$ 变为 $2\times4$ 的矩阵，也就是 $[0]$ 的意思，留下第 0 个维度，把其他维度 flatten
    3. 矩阵乘：$\sigma^x$ 作用于 $\psi'_{:,:,:,1}$ 的矩阵上
    4. 矩阵变为张量：$\phi$ 变为 $2\times2\times2$ 的张量

3. 拼接得到新 $\psi'$：

    $\psi_{:,:,:,0}' = \psi_{:,:,:,0}$
    
    $\psi_{:,:,:,1}' = \phi$
4. 指标交换，恢复原来的指标顺序

> 计算复杂度从<2 比特门作用于 3 比特量子态> 变成 <1 比特门作用于 1 比特量子态>，大幅降低了计算复杂度

TODO：看 Library.QuantumState 里的 TensorPureState 类
